# 课时21 tf.data模型简介
## 1. 理论简介
tf.data.Dataset表示一系列元素，tf.data.Dateset模块中每个元素包含一个或者多个Tensor对象。例如在图片管道中，一个元素可能是单个训练样本，具有一对表示图片数据和标签的的张量。可以通过两种不同的方式来创建tf.data.Dataset:
>1. 直接从Tensor创建Dataset，例如Dataset.from_tensor_slices()，当然numpy也是可以的，TensorFlow会自动的将其转换为Tensor；
>2. 通过对一个或者多个tf.data.Dataset对象来使用变换(例如Dataset.zip)来创建Dataset。

一个Dataset对象包含多个元素，每个元素的结构都是相同的。每个元素包含一个或者多个tf.Tensor对象，这些对象被称为组件。
Dataset的属性由构成该Dataset的元素的属性映射得到，元素可以是单个张量、张量元祖，也可以是张量的嵌套元素。

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
sb.set_style('darkgrid')
import matplotlib.pyplot as plt
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.4.0


# 2. Dataset的建立
## 2.1 一维数组的Dataset

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6, 7])
# tf.data.Dataset.zip((A, B))

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
# from_tensor_slices顾名思义就是将每个元素切片成一个组件，将其转换为tf.Tensor数据类型
for elem in dataset:
    # print(elem)
    # .numpy()方法将每个tf.Tensor转换为numpy数据类型
    print(elem.numpy())

1
2
3
4
5
6
7


## 2.2 二维数组的Dataset

In [10]:
# .from_tensor_slices要求每个数据的形状和组件是相同的
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4], [5, 6]])
dataset

<TensorSliceDataset shapes: (2,), types: tf.int32>

In [11]:
# from_tensor_slices顾名思义就是将每个元素切片成一个组件，将其转换为tf.Tensor数据类型
for elem in dataset:
    print(elem)
    # .numpy()方法将每个tf.Tensor转换为numpy数据类型
    # print(elem.numpy())

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor([5 6], shape=(2,), dtype=int32)


## 2.3 使用字典的方式建立Dataset

In [6]:
dataset_dic = tf.data.Dataset.from_tensor_slices({'a':[1, 2, 3, 4],
                                                  'b':[5, 6, 7, 8],
                                                  'c':[12, 13, 14, 15]})
dataset_dic

<TensorSliceDataset shapes: {a: (), b: (), c: ()}, types: {a: tf.int32, b: tf.int32, c: tf.int32}>

In [7]:
# from_tensor_slices顾名思义就是将每个元素切片成一个组件，将其转换为tf.Tensor数据类型
for elem in dataset_dic:
    print(elem)
    # .numpy()方法将每个tf.Tensor转换为numpy数据类型
    # print(elem.numpy())

{'a': <tf.Tensor: shape=(), dtype=int32, numpy=1>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=5>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=12>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=6>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=13>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=3>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=14>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=4>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=15>}


## 2.4 使用numpy数组的方式创建Dataset

In [63]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1, 2, 3, 4, 5, 6, 7]))
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [8]:
# Dataset提供了一种叫.take()的方法可以指定从Dataset中取出指定数目的数据
# 如果dataset是多个batch的话，take(n)则代表取出多少个batch
for elem in dataset.take(4):
    print(elem)
    # .numpy()方法将每个tf.Tensor转换为numpy数据类型
    # print(elem.numpy())

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


# 3. 对数据进行乱序

In [9]:
# buffer_size: A `tf.int64` scalar `tf.Tensor`, representing the number of elements from this dataset from which the new dataset will sample.
# .repeat(count=3)代表对打乱了的数据集重复多少次
# .batch(batch_size=4)代表每次batch生产的数据有多少个
# 一般情况下会使用到所有的数据集，所以shuffle中的参数会默认为空，repeat中的也会为空(代表一直循环重复整个数据集)，让整个数据集无限重复下去
# dataset = dataset.shuffle(buffer_size=7).repeat(count=3).batch(batch_size=4)
dataset = dataset.shuffle(buffer_size=7).repeat(count=5).batch(batch_size=4)

In [11]:
for elem in dataset:
    print(elem.numpy())

[2 7 4 6]
[1 3 5 4]
[6 3 2 1]
[5 7 7 4]
[5 2 3 1]
[6 5 2 1]
[3 6 7 4]
[5 1 4 3]
[6 2 7]


# 4. 通过map函数对数据进行操作

In [14]:
# dataset可以使用map对dataset进行快速处理，其中map中传入的是一个数据要进行处理的函数
dataset = dataset.map(tf.square)
for elem in dataset:
    print(elem.numpy())

[   6561 5764801     256   65536]
[1679616       1  390625    6561]
[1679616  390625 5764801   65536]
[    256       1 1679616   65536]
[5764801    6561  390625       1]
[    256   65536    6561 5764801]
[1679616       1     256  390625]
[1679616  390625    6561 5764801]
[    1   256 65536]
